##Imports

In [1]:
# !pip install GPUtil
# !pip install numba
# !pip install torch
# import torch
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda
# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()
#     torch.cuda.empty_cache()
#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)
#     print("GPU Usage after emptying the cache")
#     gpu_usage()
# free_gpu_cache()

In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 8.4 MB/s 
     |████████████████████████████████| 895 kB 71.3 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
     |████████████████████████████████| 6.5 MB 39.5 MB/s 
     |████████████████████████████████| 77 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install wordsegment

     |████████████████████████████████| 4.8 MB 7.0 MB/s 


In [4]:
!pip install pyyaml h5py

In [5]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 8.7 MB/s 


In [6]:
import numpy as np

import random

import os, json

import pandas as pd
pd.options.display.max_colwidth = 200

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

import transformers

from transformers import BertTokenizer, TFBertModel, TrainingArguments, Trainer
from tensorflow.keras import backend as K

from tensorflow.keras.metrics import Metric
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
%matplotlib inline

import re

import logging
tf.get_logger().setLevel(logging.ERROR)

In [7]:
tf.config.list_physical_devices('TPU')

[]

In [8]:
tf.__version__

'2.8.0'

In [9]:
transformers.__version__

'4.18.0'

##Import English Data

In [10]:
#Import data from Google drive
from google.colab import drive
drive.mount('/content/gdrive')

english_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_English/english_train.csv'
                       )

english_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_English/english_dev.csv'
                       )

english_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_English/english_test.csv'
                       )

Mounted at /content/gdrive


In [11]:
english_train

,text,label
0,"The user doing this is happy to see the gutted version on wikipedia, he is gone. Good job. Dispute is over.65.87.185.73",0
1,"That page was listed a couple of days ago, but I didn't had time to check it. Anyway, I see the problem is moot now.Tizio",0
2,"I saw that, thanks. – (talk)",0
3,Fuck you you bitch ass ho.,1
4,I really hate you \n\nAnd I want to do nasty things to you,1
...,...,...
24465,"""\n\n Don't forget the offense against non-sensible people. (Talk) """,0
24466,This isn't about Protestant IRA members. This is about YOU removing content from wikipedia and applying different set of standards to Irish Republicans than anyone else with a page on wikipedia. Y...,0
24467,"""\n\nDear Azure Citizen:\n I wrote a """"talk"""" comment about a pernicious phenomenon which has infected Wikipedia in general, and this article in particular. I didn't name anyone in particular...",0
24468,I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I...,1


In [12]:
english_dev

,text,label
0,Mortgage lending \n\nIs this the Commerce Bank that owns East-West Mortgage?,0
1,Restore Irish E-Sports page \n\nBased prior decision that this page can not be put up for speedy deletion and agreement with Jerry's decision your deletion of the Irish E-Sports page was reverte...,0
2,"Sorry, I thought no one would read it.",0
3,"You have made five edits today, all were silly and unconstructive and all have been reverted. Stop wasting everyone's time.",1
4,"""\n\nImage copyright problem with Image:AUSEPassport.jpg\n \nThanks for uploading Image:AUSEPassport.jpg. However, the image may soon be deleted unless we can determine the copyright holder and co...",0
...,...,...
3054,Argg!I just did it again!!!! (forgot to sign).,0
3055,"Wendy, get lost. Really. Not everybody shares your bigoted views. And Antonio, quite frankly, you're a fucker for comparing Mercado's sexuality to Babe Ruth's alcoholism. Get the fuck off the site...",1
3056,"""\n\n Hey Jerk \n\n""""You need to read the editnotice"""", well I did stupid. You don't have to be a jerk about it, loser.""",1
3057,Oh that is wonderful how those IP's just change around isn't it? Any other key evidence you'd like to tamper with? It sounds like you've done a lot wrong. It's really just a matter of when it catc...,0


In [13]:
english_test

,text,label
0,"Why are you still here? Can you not find anything more important to do, like killing yourself?",1
1,Its an inaccurate entry. Wikipedia should aim to be as accurate as possible.,0
2,"This bot is seriously driving us crazy\nThis image exists, I must kill it!",1
3,PLEASE STOP VANDALISING MY USER TALK PAGE,0
4,"I've removed the Orwell referrence from the influences section, as its now been a week and no response.",0
...,...,...
3054,u suck at making movies\n\nu pathetic geek\n\nur premise is shaky\n\nur outlook is bleak\n\nthere isnt a too painful way for ur life to end\n\nu really need a girlfriend,1
3055,Dont be gay - Hitler was a form of hero!!,1
3056,I wish to be block from editing Goddamit Goddamit block me from editing!,1
3057,Why did you block my page you twat. I am going to find you and me and the lads are going to play footie with your testacles. My mate told me he'd had some trouble with wankers on wikipedia some ga...,1


##Import Hindi Data

In [14]:
#Import Hindi data from Google drive
from google.colab import drive
drive.mount('/content/gdrive')

hindi_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Hindi_New/hindi_train.csv'
                       )

hindi_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Hindi_New/hindi_dev.csv'
                       )

hindi_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Hindi_New/hindi_test.csv'
                       )

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
hindi_train

,text,label
0,"RT @RishiPrasadOrg: ईश्वर किसीको प्राप्त नहीं होता, ब्रह्म किसीको प्राप्त नहीं होता बल्कि प्राप्त हुआ-सा लगता है।वह भी किसको? जिसको नहीं मि…",0
1,@BBCHindi इसे कहते हैं खतरो का खिलाड़ी।,0
2,७० साल जतन के ७ साल पतन के #IndiaCovidCrisis,0
3,"RT @AagayiNavya: चंपारण में एक चीनी मिल बंद पड़ी थी,२०१४ में मोदी ने कहा था अब यहां तभी आऊंगा जब इसी चीनी मिल की चीनी से चाय पियूंगा...\nआज…",1
4,सिर्फ़ सरकारें बदलती हैं हालात नहीं बदलते l \nकांग्रेस शासित राजस्थान की जेल में फिर एक मुसलमान \nक़ैदी की हत्या (लिंचिंग) l \n कांग्रेस शासन में राजस्थान में तीसरी लिंचिंग हो गई\nकांग्रेस सिर्फ भ...,1
...,...,...
4230,आज़ाद ख़याल वो नहि वो फूहड़ हो आज़ाद ख़याल वो जो हक़ीक़त को #बज्म,0
4231,चलो वक़्त में थोड़ा ठहर के देखे ~~ @niksism6 --&gt;&gt; my thoughts on the YourQuote app at https://t.co/PEVD4AjLw8 --&gt;&gt; Retweet If You Like this।। #niksismwrites✍️ #CoronaPandemic #...,0
4232,"RT @PihuTyagi_: लोगों के काम आते रहिये क्योंकि ,क़ुदरत का एक उसूल है कि...\n\nजिस कुंए से लोग पानी पीते रहे ,वो कभी सूखता नही है ...!!…",0
4233,"1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n17000 सिखो, 5000 भोपाली, 3000 तमिलों का कत्लेआम करवाने वाला भारत रत्न कैसे हो सके है?",1


In [16]:
hindi_dev

,text,label
0,"#ModiKaVaccineJumla जुमला वाला प्रधानमंत्री, जैसी करनी वैसी भरनी। लोग जात पात पर वोट करते हैं तो उसे भोंखने वाला कुत्ता ही मिलेगा या फिर बददिमाग गधा नाकि कोई अच्छा इंसानियत का हुनर रखने वाला नेता।",1
1,Mar javunga aaj me khushi me 😍😍,0
2,"RT @Sonam_Mumbaikar: तेरे इश्क़ में हद से गुजर जाऊं\n\nमैं झूठ कहूं तो, मर जाऊं...😍\n\n@Sonam_Mumbaikar",0
3,@Physicsgaurav Ye sale pde likhe chutiya hai 😜🤘,1
4,@RachnaSinghSP अरे चाची हो चाची.. जब सैफई में रंडी नचा के नकटेढवा देश के बर्बाद करत रहल तब तोहार जुनून कहाँ रहल हो चाची... खालीभर बकलोली करय जानेलू ये चाची.... 🤣🤣🤣🤣,1
...,...,...
524,"RT @qshaikhu: @attorneybharti भाजपा वाले तो अपना टेबल भी लेकर निकल लिए, फ्रूटी की खाली बोटल छोड़कर, स्वच्छ भारत मैं अपना योगदान देकर । htt…",1
525,"RT @BJP4India: भ्रष्टाचार हो, महंगाई हो या 1984 में हजारों सिखों का कत्ल हुआ हो इनका जवाब यही होता है- 'हुआ तो हुआ'।\n\nजिस अहंकार में ये लोग…",1
526,"शुक्रिया इमरान प्रताप गढ़ी साहेब शुक्रिया अमनातुल्लह साहेब ,, इस दुख की घड़ी में ओसामा शहाब का साथ देने के लिए साथ खड़े होने के लिए,,उन लोगों का भी बहुत शुक्रिया जो दिल्ली में ओसामा भाई के साथ खड...",0
527,#bengalviolence Be careful everyone. Some unscrupulous people are posting these pictures on social media to spread tension in the state and the country. #CovidIndia #COVIDEmergencyIndia #ममता_की_...,0


In [17]:
hindi_test

,text,label
0,"@zishanAliRJD @iAnantSingh_ *ओसामा साहब ने सिर्फ उस ट्वीट और ट्विटर अकाउंट को फर्जी बताया है ना कि राजद का पक्ष लिए और ना की राजद का गुणगान किये है, संयम रखिए जल्द ही सब कुछ सामने आएगा।* #JusticeF...",0
1,"@China_Amb_India @narendramodi I am shocked they didn't mention a line (Karam, you have been our worst sufferer) (करम हमारे ख़राब ह भुगत आप रहे ह) #wohanvirus #chinesevirus",0
2,कल से 18 से ऊपर वालो को हवा की वैक्सीन लगेगी... #ModiKaVaccineJumla,0
3,इधर की बात उधर करने में \nआज भी जीमेल से आगे है \nफीमेल!!!,0
4,☯️ मोदी जी ➡️ सोंगंद मुझे इस मिट्टी की मै देश नहीं मिटने दूंगा मै देश नहीं झुकने दूंगा !! और हमने भैंचोद ये बात सीरियस लेली!!!! ले कोरॉना➡️ सालो अब भुगतो 👹 #ModiMustResign #ModiKaVaccineJumla,1
...,...,...
525,"हे प्रभु!! निंदा????? नेताओं की , और सरकारों के निकम्मेपन को छुपाने के लिए एक शब्द काफी है ""निंदा"" । #BengalBurning https://t.co/FcFx0eTSW0",1
526,"अगर आपको पाकिस्तान या बांग्लादेश के हिंदुओं की चिंता होती है की उनके साथ वहा गलत हो रहा है और आप उनको भारत बुलाना चाहते हो, भारत बुला के भी करोगे क्या? भारत के हिंदुओ का घर लूटा जा रहा है, हिंदू म...",1
527,RT @SHIVANI92_: क्या ऐसा करना चाहिए? अपना विचार जरुर कमेंट करें।\n@cp4bhai @MLArajeshSP @ShivaYa36977421 @Narendryadav01 @0359R @AshishS5185…,0
528,RT @pravinj14696933: राजीव ने बोफोर्स खाया हैं\nसोनिया ने खाया था अगस्ता\nवाड्रा ने ज़मीन हड़पी हैं\nराहुल ने हेरॉल्ड डकारा थां\n\nइंदिरा ने इमर…,1


##Import Korean Data

In [18]:
#Import Korean data from Google drive
from google.colab import drive
drive.mount('/content/gdrive')

korean_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Korean_New/korean_train.csv'
                       )

korean_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Korean_New/korean_dev.csv'
                       )

korean_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Korean_New/korean_test.csv'
                       )

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
korean_train

,text,label
0,불쌍해 보이는 이윤 뭘까?~,0
1,독과점의 결과,0
2,별 시덥지않은 악플들은 모냐,0
3,사랑의 불시착 하는시간인데 이상한 노잼 드라마가 하고있다 ㅡㅡ 어딨는거냐 표치수~~~~~,1
4,저밖에몰라..남은사람 어쩌라고. 참 이기적이네,0
...,...,...
5828,다음 생에는 중국 남자 만나야지,0
5829,난 왜 이부진이 생각나는걸까?,0
5830,4년전정준영카톡이 이게 이렇게 큰일이냐?ㅋ 더럽네정말.. 누가봐도 꼬리짜르기구만,1
5831,악플러 쓰레기들 많네,1


In [20]:
korean_dev

,text,label
0,다 그러고 애낳고 키웠고 다 그러고 산다예전에 우리엄마들은 어떻게 애키우며 밭일하고 살았는지...다 본인이 선택한 인생이고 힘든거 이상의 기쁨이 있다기자들도 이런거 퍼날리지말고 발로뛴 기사나 써라,1
1,이제 별 감흥도 없는 애를 멀...,1
2,홍상수 김민희 좋아하는 감독도 배우도 아니지만 남여관계 모르는 거다. 이렇게 비난받을 일은 아닌듯..,0
3,이쁘게 컸네^^,0
4,아니 저 금동현 자리에 진우가 있어야 한다며 ??,0
...,...,...
724,에이프릴 이나은 고정 해도될듯 너무잘하더라♡,0
725,다니엘 잘못한거 없어요 공룡과 힘겨운 분쟁할때당신을 행복하게 해줄 사람 만나는 거ㅡ무조건 응원합니다,0
726,새로운 남자친구나 남편은 저 사이가 얼마나 불편할까,1
727,수아가 불쌍하다 납치되서 20년을 힘들게 살고 남자놈은 수아버리고 민채린한테 가서 다리에서 뛰려고했다 수아가 저둘때메 눈안뒤집히겠냐 그리고 수아정체알고도 부모와못만나게 한채린이를 알게된다면 그집서쫒겨나야죠,0


In [21]:
korean_test

,text,label
0,"팀으로 데뷔한거면 개인활동 했어도 N빵 해야지... 그게 팀을 위해서도 맞는거고~~ 근데 여기 댓글은 다 왜 나누냐는 식이네, 당장 혼자만 배불러야 된다는 심리들인건지, 팀이 유지되야 인기도 오래갈텐데",0
1,왕지혜 34살이지만 외모는 인정한다.여자라면 이정도는 되야지,0
2,여자들이 80프로잉 악플 남자는 여자욕 안해,1
3,설현이 떨고 있다... 아니겠지 아닐거야 그것만은 안돼,0
4,다된 기생충 잔치에재 뿌린 방가방송과 안현모 다신 나서지 말자,1
...,...,...
725,유재석 재벌인디,0
726,보컬만 아니면 상관없잖아..?,0
727,셔누가 무슨 잘뭇했나요? 결백하다...,0
728,사귐 사귄다고 말함되지 참 답답따,0


##Import Translated Data

In [22]:
#Import Translated data from Google drive
from google.colab import drive
drive.mount('/content/gdrive')

korean_translated_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Translated_Google_Translate/korean_translated_train.csv'
                       )

korean_translated_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/korean_translated_dev.csv'
                       )

korean_translated_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/korean_translated_test.csv'
                       )

hindi_translated_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Translated_Google_Translate/hindi_translated_train.csv'
                       )

hindi_translated_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/hindi_translated_dev.csv'
                       )

hindi_translated_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/hindi_translated_test.csv'
                       )


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
korean_translated_train

,text,label
0,What is the poor-looking profit?,0
1,result of monopoly,0
2,Are there any malicious comments that aren't too bitter?,0
3,"It's time for a love crash, but a strange no-jam drama is being played ㅡㅡ Where are you? Pyo Chi-soo~~~~~",1
4,I don't know.. What about the rest of me? It's so selfish,0
...,...,...
5828,I need to meet a Chinese man in my next life,0
5829,Why am I thinking of Lee Boo-jin?,0
5830,4 years ago Jung Joon-young Kakao Talk is this such a big deal?,1
5831,There are a lot of malicious comments,1


In [24]:
korean_translated_dev

,text,label
0,"After all, they had children and raised them, and they all live like that. In the past, how my mothers raised children and worked in the fields... They are all their own choices, and there is joy...",1
1,Far from a kid who doesn't have much inspiration anymore...,1
2,"Hong Sang-soo and Kim Min-hee I am neither a favorite director nor an actress, but I do not know the relationship between men and women. I don't think it's something to be criticized for.",0
3,You grew up pretty ^^,0
4,"No, he said that Jinwoo should be in the place of Geum Dong-hyun??",0
...,...,...
724,April's Naeun looked so good that it could be fixed♡,0
725,Daniel didn't do anything wrong To meet someone who will make you happy when you have a difficult dispute with a dinosaur - I support you unconditionally,0
726,How uncomfortable would my new boyfriend or husband be with me?,1
727,"feel sorry for Soo-ah, I was kidnapped and I had a hard time living for 20 years. The man left Su-ah and went to Min Chae-rin to run on the bridge. Won't you turn your eyes when Su-a is gone? it'...",0


In [25]:
korean_translated_test

,text,label
0,"If you debuted as a team, even if you did individual activities, you should be N bread... That's right for the team too~~ But it's like why are you sharing all the comments here, maybe it's the m...",0
1,"Wang Ji-hye is 34 years old, but she admits her looks. If it's a woman, it should be like this",0
2,"Women get 80% of malicious comments, and men don't lust for women.",1
3,Seolhyun is trembling... no no no no no,0
4,"Banga Broadcasting and Ahn Hyun-mo, who sprinkled ashes on the exhausted parasite feast, let's not go out again",1
...,...,...
725,Yoo Jae-seok chaebol indie,0
726,It doesn't matter if it's not vocals...?,0
727,What did Shownu do well? innocent...,0
728,"They say they're dating, so it's frustrating",0


In [26]:
hindi_translated_train

,text,label
0,"RT @RishiPrasadOrg: God is not attained by anyone, Brahman is not attained by anyone but seems to have been achieved. To whom too? Who doesn't get…",0
1,@BBCHindi It is called Khatron Ka Khiladi.,0
2,"70 years of saving, 7 years of downfall #IndiaCovidCrisis",0
3,"RT @AagayiNavya : A sugar mill was closed in Champaran, in 2014 Modi had said that now I will come here only when I will drink tea from the sugar of this sugar mill… today…",1
4,"Only the governments change, the situation does not change. Again a Muslim prisoner was murdered (lynching) in the jail of Congress-ruled Rajasthan. The third lynching took place in Rajasthan unde...",1
...,...,...
4230,"Free thoughts are not those who are stupid, free thoughts are those who destroy reality.",0
4231,Let's stop in time and see ~~ @niksism6 ---&gt;&gt; my thoughts on the YourQuote Retweet If You Like this.. #niksismwrites✍️ #CoronaPandemic #Bazm #Shayari #IndiaCovidCrisis https://t.co/PAedmxvIZo,0
4232,"RT @PihuTyagi_: Keep coming to people's work because, there is a principle of nature that... the well from which people keep drinking water, it never dries up...!!…",0
4233,"Godse could be a terrorist by killing 1 man, so how could the one who killed 17000 Sikhs, 5000 Bhopalis, 3000 Tamils become Bharat Ratna?",1


In [27]:
hindi_translated_dev

,text,label
0,"#ModiKaVaccineJumla Prime Minister with a jumla, do whatever you want to do. If people vote on the basis of caste, then he will get only a barking dog or a bad minded donkey and not a leader with...",1
1,Mar javunga aaj me khushi me,0
2,"RT @Sonam_Mumbaikar: If I tell lies, I will go beyond my limits...😍 @Sonam_Mumbaikar",0
3,@Physicsgaurav Ye sale pde likhe chutiya hai,1
4,"@RachnaSinghSP Hey aunty ho aunty.. when you are ruining the country of Randi Nacha in Saifai, then where is your passion?",1
...,...,...
524,"RT @qshaikhu : @attorneybharti BJP people even took their tables and left empty bottles of frutti, contributing to clean India. htt…",1
525,"RT @BJP4India: Be it corruption, inflation or the killing of thousands of Sikhs in 1984, the answer is 'hua to hua'. The arrogance in which these people…",1
526,"Thank you Imran Pratap Gadhi Saheb Thank you Amnatollah Saheb, For standing together to support Osama Shahab in this sad hour, Many thanks also to those who stood with Osama Bhai in Delhi, Bihar ...",0
527,#bengalviolence Be careful everyone. Some unscrupulous people are posting these pictures on social media to spread tension in the state and the country. #CovidIndia #COVIDEmergencyIndia #Mamata_Ki...,0


In [28]:
hindi_translated_test

,text,label
0,"@zishanAliRJD @iAnantSingh_ *Osama sahib has just called that tweet and twitter account fake, not favoring RJD and not praising RJD, have patience, everything will come out soon.* #JusticeForShah...",0
1,"@China_Amb_India @narendramodi I am shocked they didn't mention a line (Karam, you have been our worst sufferer) (Karam, you have been our worst sufferer) #wohanvirus #chinesevirus",0
2,"From tomorrow, people above 18 will get air vaccine... #ModiKaVaccineJumla",0
3,"Even today, female is ahead of Gmail in talking about it here and there!!!",0
4,"️ Modi ji ️ Songand I will not allow the country to be destroyed by this soil, I will not let the country bow down !! And we took this thing seriously!!!! Le Corona ️ Years now suffer #ModiMustRes...",1
...,...,...
525,Oh God!! Condemnation????? One word is enough to hide the inefficiency of politicians and governments. #BengalBurning https://t.co/FcFx0eTSW0,1
526,"If you are worried about the Hindus of Pakistan or Bangladesh that there is wrong with them and you want to call them to India, what will you do by calling them to India? The houses of Hindus of I...",1
527,RT @SHIVANI92_: Should it be done? Do comment your thoughts. @cp4bhai @MLArajeshSP @ShivaYa36977421 @Narendryadav01 @0359R @AshishS5185…,0
528,"RT @pravinj14696933: Rajiv has eaten Bofors, Sonia has eaten Agusta Vadra has grabbed land, Rahul has Harold Dakara, Indira has Immar…",1


##Tokenize Data

In [29]:
# # Function to pre-process tweets


# def clean_tweet(tweet):
#   tweet = re.sub(r"\B@\w+", "USERNAME", tweet) #Replace Username
#   tweet = re.sub(r"http\S+", "", tweet) # Remove url

#   hashtags = re.findall(r"(#\w+)", tweet)
#   for hs in hashtags:
#     words = " ".join(ws.segment(hs))
#     if len(words) > 0:
#       tweet = tweet.replace(hs, words)
#     else:
#       tweet = tweet.replace(hs, hs[1:])
  
#   return tweet


In [30]:
# Specify tokenzer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
mbert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [31]:
#Tokenize text, see next cell for choice of max_length
max_length = 512



# English
english_x_train = tokenizer(list(english_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
english_y_train = english_train.label

english_x_dev = tokenizer(list(english_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
english_y_dev = english_dev.label

english_x_test = tokenizer(list(english_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
english_y_test = english_test.label

In [32]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max length can be *just* more than this
# max tokenized length is 512 :( )
# Choosing 512 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, english_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, english_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, english_x_test.input_ids.numpy()).max())

512
512
512


In [33]:
#Tokenize Hindi text, see next cell for choice of max_length
max_length = 512



#Hindi
hindi_x_train = tokenizer(list(hindi_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_y_train = hindi_train.label

hindi_x_dev = tokenizer(list(hindi_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_y_dev = hindi_dev.label

hindi_x_test = tokenizer(list(hindi_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_y_test = hindi_test.label

In [34]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max olength can be *just* more than this
# Choosing 225 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, hindi_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, hindi_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, hindi_x_test.input_ids.numpy()).max())

220
170
181


In [35]:
#Tokenize Hindi text, see next cell for choice of max_length
max_length = 512



#Korean
korean_x_train = tokenizer(list(korean_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_y_train = korean_train.label

korean_x_dev = tokenizer(list(korean_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_y_dev = korean_dev.label

korean_x_test = tokenizer(list(korean_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_y_test = korean_test.label

In [36]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max olength can be *just* more than this
# Choosing 225 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, korean_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, korean_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, korean_x_test.input_ids.numpy()).max())

94
95
91


In [37]:
#Import Translated data from Google drive
from google.colab import drive
drive.mount('/content/gdrive')

korean_translated_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Translated_Google_Translate/korean_translated_train.csv'
                       )

korean_translated_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/korean_translated_dev.csv'
                       )

korean_translated_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/korean_translated_test.csv'
                       )

hindi_translated_train = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)/Data_Translated_Google_Translate/hindi_translated_train.csv'
                       )

hindi_translated_dev = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/hindi_translated_dev.csv'
                       )

hindi_translated_test = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks (1)//Data_Translated_Google_Translate/hindi_translated_test.csv'
                       )


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
korean_translated_train

,text,label
0,What is the poor-looking profit?,0
1,result of monopoly,0
2,Are there any malicious comments that aren't too bitter?,0
3,"It's time for a love crash, but a strange no-jam drama is being played ㅡㅡ Where are you? Pyo Chi-soo~~~~~",1
4,I don't know.. What about the rest of me? It's so selfish,0
...,...,...
5828,I need to meet a Chinese man in my next life,0
5829,Why am I thinking of Lee Boo-jin?,0
5830,4 years ago Jung Joon-young Kakao Talk is this such a big deal?,1
5831,There are a lot of malicious comments,1


In [39]:
korean_translated_dev

,text,label
0,"After all, they had children and raised them, and they all live like that. In the past, how my mothers raised children and worked in the fields... They are all their own choices, and there is joy...",1
1,Far from a kid who doesn't have much inspiration anymore...,1
2,"Hong Sang-soo and Kim Min-hee I am neither a favorite director nor an actress, but I do not know the relationship between men and women. I don't think it's something to be criticized for.",0
3,You grew up pretty ^^,0
4,"No, he said that Jinwoo should be in the place of Geum Dong-hyun??",0
...,...,...
724,April's Naeun looked so good that it could be fixed♡,0
725,Daniel didn't do anything wrong To meet someone who will make you happy when you have a difficult dispute with a dinosaur - I support you unconditionally,0
726,How uncomfortable would my new boyfriend or husband be with me?,1
727,"feel sorry for Soo-ah, I was kidnapped and I had a hard time living for 20 years. The man left Su-ah and went to Min Chae-rin to run on the bridge. Won't you turn your eyes when Su-a is gone? it'...",0


In [40]:
korean_translated_test

,text,label
0,"If you debuted as a team, even if you did individual activities, you should be N bread... That's right for the team too~~ But it's like why are you sharing all the comments here, maybe it's the m...",0
1,"Wang Ji-hye is 34 years old, but she admits her looks. If it's a woman, it should be like this",0
2,"Women get 80% of malicious comments, and men don't lust for women.",1
3,Seolhyun is trembling... no no no no no,0
4,"Banga Broadcasting and Ahn Hyun-mo, who sprinkled ashes on the exhausted parasite feast, let's not go out again",1
...,...,...
725,Yoo Jae-seok chaebol indie,0
726,It doesn't matter if it's not vocals...?,0
727,What did Shownu do well? innocent...,0
728,"They say they're dating, so it's frustrating",0


In [41]:
hindi_translated_train

,text,label
0,"RT @RishiPrasadOrg: God is not attained by anyone, Brahman is not attained by anyone but seems to have been achieved. To whom too? Who doesn't get…",0
1,@BBCHindi It is called Khatron Ka Khiladi.,0
2,"70 years of saving, 7 years of downfall #IndiaCovidCrisis",0
3,"RT @AagayiNavya : A sugar mill was closed in Champaran, in 2014 Modi had said that now I will come here only when I will drink tea from the sugar of this sugar mill… today…",1
4,"Only the governments change, the situation does not change. Again a Muslim prisoner was murdered (lynching) in the jail of Congress-ruled Rajasthan. The third lynching took place in Rajasthan unde...",1
...,...,...
4230,"Free thoughts are not those who are stupid, free thoughts are those who destroy reality.",0
4231,Let's stop in time and see ~~ @niksism6 ---&gt;&gt; my thoughts on the YourQuote Retweet If You Like this.. #niksismwrites✍️ #CoronaPandemic #Bazm #Shayari #IndiaCovidCrisis https://t.co/PAedmxvIZo,0
4232,"RT @PihuTyagi_: Keep coming to people's work because, there is a principle of nature that... the well from which people keep drinking water, it never dries up...!!…",0
4233,"Godse could be a terrorist by killing 1 man, so how could the one who killed 17000 Sikhs, 5000 Bhopalis, 3000 Tamils become Bharat Ratna?",1


In [42]:
hindi_translated_dev

,text,label
0,"#ModiKaVaccineJumla Prime Minister with a jumla, do whatever you want to do. If people vote on the basis of caste, then he will get only a barking dog or a bad minded donkey and not a leader with...",1
1,Mar javunga aaj me khushi me,0
2,"RT @Sonam_Mumbaikar: If I tell lies, I will go beyond my limits...😍 @Sonam_Mumbaikar",0
3,@Physicsgaurav Ye sale pde likhe chutiya hai,1
4,"@RachnaSinghSP Hey aunty ho aunty.. when you are ruining the country of Randi Nacha in Saifai, then where is your passion?",1
...,...,...
524,"RT @qshaikhu : @attorneybharti BJP people even took their tables and left empty bottles of frutti, contributing to clean India. htt…",1
525,"RT @BJP4India: Be it corruption, inflation or the killing of thousands of Sikhs in 1984, the answer is 'hua to hua'. The arrogance in which these people…",1
526,"Thank you Imran Pratap Gadhi Saheb Thank you Amnatollah Saheb, For standing together to support Osama Shahab in this sad hour, Many thanks also to those who stood with Osama Bhai in Delhi, Bihar ...",0
527,#bengalviolence Be careful everyone. Some unscrupulous people are posting these pictures on social media to spread tension in the state and the country. #CovidIndia #COVIDEmergencyIndia #Mamata_Ki...,0


In [43]:
hindi_translated_test

,text,label
0,"@zishanAliRJD @iAnantSingh_ *Osama sahib has just called that tweet and twitter account fake, not favoring RJD and not praising RJD, have patience, everything will come out soon.* #JusticeForShah...",0
1,"@China_Amb_India @narendramodi I am shocked they didn't mention a line (Karam, you have been our worst sufferer) (Karam, you have been our worst sufferer) #wohanvirus #chinesevirus",0
2,"From tomorrow, people above 18 will get air vaccine... #ModiKaVaccineJumla",0
3,"Even today, female is ahead of Gmail in talking about it here and there!!!",0
4,"️ Modi ji ️ Songand I will not allow the country to be destroyed by this soil, I will not let the country bow down !! And we took this thing seriously!!!! Le Corona ️ Years now suffer #ModiMustRes...",1
...,...,...
525,Oh God!! Condemnation????? One word is enough to hide the inefficiency of politicians and governments. #BengalBurning https://t.co/FcFx0eTSW0,1
526,"If you are worried about the Hindus of Pakistan or Bangladesh that there is wrong with them and you want to call them to India, what will you do by calling them to India? The houses of Hindus of I...",1
527,RT @SHIVANI92_: Should it be done? Do comment your thoughts. @cp4bhai @MLArajeshSP @ShivaYa36977421 @Narendryadav01 @0359R @AshishS5185…,0
528,"RT @pravinj14696933: Rajiv has eaten Bofors, Sonia has eaten Agusta Vadra has grabbed land, Rahul has Harold Dakara, Indira has Immar…",1


In [44]:
#Tokenize Hindi text, see next cell for choice of max_length
max_length = 512



#Hindi
hindi_translated_x_train = tokenizer(list(hindi_translated_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_translated_y_train = hindi_translated_train.label

hindi_translated_x_dev = tokenizer(list(hindi_translated_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_translated_y_dev = hindi_translated_dev.label

hindi_translated_x_test = tokenizer(list(hindi_translated_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_translated_y_test = hindi_translated_test.label

In [45]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max olength can be *just* more than this
# Choosing 225 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, hindi_translated_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, hindi_translated_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, hindi_translated_x_test.input_ids.numpy()).max())

180
172
171


In [46]:
#Tokenize Korean Translated text, see next cell for choice of max_length
max_length = 512

#Korean
korean_translated_x_train = tokenizer(list(korean_translated_train.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_translated_y_train = korean_translated_train.label

korean_translated_x_dev = tokenizer(list(korean_translated_dev.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_translated_y_dev = korean_translated_dev.label

korean_translated_x_test = tokenizer(list(korean_translated_test.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_translated_y_test = korean_translated_test.label

In [47]:
# Max number Non zero elements in input_ids, i.e. longest tokenized sentence for each set, max olength can be *just* more than this
# Choosing 225 in cell above

print(np.apply_along_axis(np.count_nonzero, 1, korean_translated_x_train.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, korean_translated_x_dev.input_ids.numpy()).max())
print(np.apply_along_axis(np.count_nonzero, 1, korean_translated_x_test.input_ids.numpy()).max())

101
90
101


##Define Model Basics

In [48]:
# Custom Metrics
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [49]:
metric = tfa.metrics.F1Score(num_classes=2, threshold=0.5, average = 'weighted')


In [50]:
def create_classification_model(hidden_sizes = (256, 128), 
                                train_layers = -1, 
                                optimizer=tf.keras.optimizers.Adam(),
                                hidden_initializer = 'he_uniform',
                                dropout = 0):
    """
    Build a simple classification model with mBERT.
    """

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}


    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in mbert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = mbert_model(bert_inputs)


    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])


    hidden = tf.keras.layers.Dense(hidden_sizes[0], name='hidden_layer', activation='relu', kernel_initializer = hidden_initializer)(classification_token)
    hidden = tf.keras.layers.LayerNormalization()(hidden)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    hidden = tf.keras.layers.Dense(hidden_sizes[1], name='hidden_layer_2', activation='relu')(hidden)
    hidden = tf.keras.layers.LayerNormalization()(hidden)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    classification_model.compile(optimizer=optimizer,
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                            metrics=['acc',f1_m,precision_m, recall_m])


    return classification_model

##Reload Models and Test on Untranslated Data

###Hindi Baseline mBERT

In [73]:
# reload saved model for Hindi (baseline mBERT)

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/Hindi_mBERT_Baseline/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rate = .001
dropout = 0
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

y_pred = reloaded_model.predict(x=[hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask], 
                        )

print('Dev Set F-1 Score: {:.3f}'.format(f1_score(hindi_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score: {:.3f}'.format(f1_score(hindi_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[hindi_x_test.input_ids, hindi_x_test.token_type_ids, hindi_x_test.attention_mask], 
                        )

print('Test Set F-1 Score: {:.3f}'.format(f1_score(hindi_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score: {:.3f}'.format(f1_score(hindi_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))


Dev Set F-1 Score: 0.714
Dev Set Macro F-1 Score: 0.616
Test Set F-1 Score: 0.699
Test Set Macro F-1 Score: 0.606


In [52]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [66]:
def TP_FP(y_actual, y_hat):
  TP_FP = []
  for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP_FP.append('TP')
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           TP_FP.append('FP')
        if y_actual[i]==y_hat[i]==0:
           TP_FP.append('TN')
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           TP_FP.append('FN')

  return TP_FP

In [74]:
pd.DataFrame({'mBERT_predicted_prob':np.concatenate(y_pred), 
              'predicted label':np.concatenate(y_pred>0.5).astype(int),
              'TP_FP_mBERT':TP_FP(hindi_y_test, np.concatenate(y_pred>0.5).astype(int))}).to_excel('hindi.xlsx')

In [75]:
!ls

drive  gdrive  hindi.xlsx  korean.xlsx	sample_data


In [77]:
from google.colab import files
files.download('hindi.xlsx') 
files.download('korean.xlsx') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###Extract Examples for Error Analysis

In [ ]:
Analysis = pd.DataFrame(data = {'predicted_prob':y_pred.reshape(hindi_y_test.shape), 'predicted_class':(y_pred>0.5).reshape(hindi_y_test.shape).astype(int),'true_class':hindi_y_test})

In [ ]:
Analysis

,predicted_prob,predicted_class,true_class
0,0.347836,0,0
1,0.325049,0,0
2,0.356229,0,0
3,0.808870,1,0
4,0.441667,0,1
...,...,...,...
525,0.507423,1,1
526,0.508016,1,1
527,0.567495,1,0
528,0.651302,1,1


In [ ]:
Analysis['correct_prediction'] = Analysis.predicted_class == Analysis.true_class

In [ ]:
Analysis.reset_index(drop=True, inplace=True)

In [ ]:
Analysis

,predicted_prob,predicted_class,true_class,correct_prediction
0,0.347836,0,0,True
1,0.325049,0,0,True
2,0.356229,0,0,True
3,0.808870,1,0,False
4,0.441667,0,1,False
...,...,...,...,...
525,0.507423,1,1,True
526,0.508016,1,1,True
527,0.567495,1,0,False
528,0.651302,1,1,True


In [ ]:
# Correctly classified as toxic with the largest probability
pd.set_option('display.max_colwidth', None)

temp_df = Analysis[(Analysis.correct_prediction==True) & (Analysis.true_class == 1)].nlargest(15, ['predicted_prob'])

pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

,text,label,predicted_prob,correct_prediction
441,@iashwathama इनको ऐसे ही लोग नेता बनाने के लिए मिलेंगे। खुद क्या हैं ये टोंटीचोर।,1,0.910769,True
410,@ishwars03962227 @Aasif_Bhandari @anamikaa_143 क्यों तेरी दी Di के साथ सात फेरे लेके गद्दारी कर दी क्यों बे गुफा पुत्र नकली हिन्दू बोल आगे साले गद्दार देशद्रोही माफी वीर की नाजायज़ औलाद गद्दारी का इतिहास है तुम गुफा पुत्रो का,1,0.907276,True
44,@TeamSaath @DasnaMandir_ @TwitterSpaces @Twitter @NarsinghUvach कांग्रेसी कुत्तो को अपने बापो के खिलाफ सुन्ना पसन्द नही है। विदेशी कुतिया के गुलाम बन चुके है ये हरामजादे,1,0.904074,True
49,@SheilaDikshit @ArvindKejriwal सठिया गया है केजरी,1,0.900834,True
494,@kanpur62 Itti जल्दी थोड़ी मरेगा बुड्ढा,1,0.898790,True
267,"@LambaAlka तुम्हारा रिस्तेदार आतंकी घोसित हो गया हल्का .....थोरा सा रो लो , नहीं तो पेमेंट नहीं मिलेगा ।",1,0.880756,True
331,@DrMashurGulatig @IndianSoldier_1 खुजलीवाल आया होगा सर....बहुत दिन से उसकी खांसी बंद है ना,1,0.880125,True
140,दो कोड़ी का टीकट 6 करोड़ में बेच कर केजरीवाल जी सिद्द कर दिया की उनसे बड़ा बनिया कोई नही,1,0.879009,True
352,"@imjaiswalritik इस भाई को क्या तकलीफ है बवासीर हो गई है क्या या #Islamophobia , का कीडा गांड़ मे काट रहा है",1,0.877579,True
107,@ArvindKejriwal बड़े बुज़ुर्ग का सम्मान उस दिन कहाँ चला गया था जब शीला जी को गलियाँ दे रहे थे बहरे बाज़ार में। कितना उलटा होगे सड़ जी कहीं तो रुको बे,1,0.876861,True


In [ ]:
error_analysis_hindi = pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1).text

In [ ]:
# Correctly classified as non-toxic with the lowest probability (of toxic)
pd.set_option('display.max_colwidth', None)

temp_df = Analysis[(Analysis.correct_prediction==True) & (Analysis.true_class == 0)].nsmallest(15, ['predicted_prob'])

pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

,text,label,predicted_prob,correct_prediction
456,#25सांसद_लापता_है #ResignModi #ModiKaVaccineJumla #ModiMustResign #FarmLaws_NextPandemic https://t.co/wdfjiImSe3,0,0.154396,True
212,@OmarAbdullah मोदी तुम इस्तीफा दो हम तुम्हारे साथ है #ResignModi #Resign_PM_Modi #ModiHataoDeshBachao https://t.co/spsvPTov8W,0,0.161979,True
172,#ModiKaVaccineJumla #ModiResign #IndiaFightsCorona #PMCaresFund_का_हिसाब_दो #PMDoesNotCare https://t.co/FxEwHIH6KI,0,0.167698,True
353,शहाबुद्दीन जी को न्याय मिलना चाहिए #JusticeForShahabuddin,0,0.175670,True
116,आज़म खान साहब का इलाज AIIMS में कराया जाए… शहाबुद्दीन साहब की तरह उन्हें सिस्टम भरोसे नही छोड़ सकते😢🙏 #JusticeForShahabuddin,0,0.179889,True
40,आप सब लिखते रहिए जब तक ये हैशटैग नम्बर 1 ना पहुंच जाए.. #JusticeForShahabuddin,0,0.180384,True
270,जो शहाबुद्दीन के साथ हुआ वो आज़म खान के साथ नहीं होना चाहिए इसलिए आइए हम सब आवाज उठाते हैं #JusticeForShahabuddin,0,0.181141,True
258,#कृषि_कानून_रद्द_करने_पड़ेंगे #कृषि_कानून_रद्द_करने_पड़ेंगे #Resign_PM_Modi #ResignModi #FarmersProtest #BanEVM_SaveDemocracy #Ban_Evm #कृषि_कानून_रद्द_करने_पड़ेंगे https://t.co/f3Ia3QEOUM,0,0.182063,True
100,Trending now. #नरेंद्र_मोदी_ग्लोबल_पप्पू_है #मोदी_भगाओ_देश_बचाओ #मोदी_इस्तीफा_दो #ResignModi #ModiMustResign https://t.co/iJp5YF5FHP,0,0.189003,True
178,#OPS #BanEVM #पुरानी_पेंशन_बहाल_करो #FarmersProtest #SpeakUpForFarmers #StopPrivatization #ModiMadeDisaster #NPS_भारत_छोड़ो #NPS_QUIT_INDIA #NPSनिजीकरणभारतछोड़ो #OPS_कर्मचारी_का_अधिकार #NPSनिजीकरणकेख़िलाफ़_इंक़लाब #स्वास्थ्य_क्षेत्र_मे_बजट_बढ़ाओ #Resign_PM_Modi #Vaccine_और_MSP_दो https://t.co/NKfTIqNjO9,0,0.202640,True


In [ ]:
error_analysis_hindi = pd.concat([error_analysis_hindi , pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1).text], axis = 0)

In [ ]:
# incorrectly classified as toxic with the highest probability (of toxic)
pd.set_option('display.max_colwidth', None)


temp_df = Analysis[(Analysis.correct_prediction==False) & (Analysis.true_class == 0)].nlargest(15, ['predicted_prob'])

pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

,text,label,predicted_prob,correct_prediction
437,@Vyom127 @desimojito chalega paise wala hai,0,0.892726,False
205,बेटा पहले आपनी नौकरी तो कर ले वरना राहुल बाबा तेरी पगार काट लेगे,0,0.884157,False
144,@TajinderBagga पा जी लोडे ला ते । चक ते फट्टे । फतेह ही फतेह ।,0,0.857700,False
243,"चुनाव प्रचार में सनी देओल हेंडपंप लेकर प्रचार कर रहा है, ओर कुछ भाजपा मूर्ख कह रहै है, कि यह वही हेंडपंप है जो पाकिस्तान में उखाड़ा था और अंधभक्त इस की बात मान भी रहे हैं, बस में जानकारी के लिए बता दूं कि फिल्म गदर में जो हेंडपंप लखनऊ में उखाड़ा गया था, पाकिस्तान में नहीं!",0,0.856919,False
224,अब क्या ऐसे आलसी लोगो के लिए भी मोदीजी दोषी है ?? 😉😉😉😉,0,0.850026,False
230,@Kavita_Sharma4 लगता चेक कर लिया हे आप ने उनके पास जा के वसे किसके सात गये थे एक का नाम बताना जभि पता चलेगा वो हिजड़ा हे।,0,0.845854,False
25,@Arvinderghai @jigneshmevani80 बरनोल लगाकर देखा है क्या।हैं चौकीदार जो है,0,0.832100,False
247,@dkkbsr यह भारत का दुर्भाग्य है कि भारत में ऐसे नेता और ऐसे अधिकारी है जो इन अवैध मजारों को देख कर भी अनदेखा कर देते हैं @narendramodi @AmitShah @LtGovDelhi @AshwiniUpadhyay लेकिन साधारण व्यक्ति प्लॉट लेकर मकान बनाने लगे तो यह हतोड़ा लेकर पहुंच जाते हैं समस्त कानून लागू हो कर देते हैं,0,0.828681,False
359,@NitishKumar प्लेस्मेंट कितना हुआ ? नेतागीरी और फूहड़ भोजपुरी फ़िल्म उद्योग के अलावा बिहारियों को आप लोगों ने दिया हीं क्या है! 🤔🤔,0,0.826488,False
96,@VijayVst0502 इसपे #SupremeCourt स्वतः संज्ञान क्यों नही लेती है ? @PMOIndia @HMOIndia @rsprasad एक प्रयास सरकार द्वारा शुरू हो ! भारत वर्ष में सनातन धर्म को दान में मिले जमीन की जानकारी जुटाई जाए यदि किसी प्रकार से कोई अवैध कब्जा है तो उसे सरकार की निगरानी में वापस लिया जाए,0,0.820995,False


In [ ]:
error_analysis_hindi = pd.concat([error_analysis_hindi , pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1).text], axis = 0)

In [ ]:
# incorrectly classified as non-toxic with the lowest probability (of toxic)
pd.set_option('display.max_colwidth', None)


temp_df = Analysis[(Analysis.correct_prediction==False) & (Analysis.true_class == 1)].nsmallest(15, ['predicted_prob'])

pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

,text,label,predicted_prob,correct_prediction
218,आज़म खान साहब का इलाज AIIMS में कराया जाए… शहाबुद्दीन साहब की तरह उन्हें सिस्टम भरोसे नही छोड़ सकते😢🙏 #JusticeForShahabuddin,1,0.179889,False
360,अरे मुसलमानो कब तक मरते रहोगे #JusticeForShahabuddin,1,0.183697,False
123,"जो लोग मुसलमानों के मसीहा बने बैठे थे, एक एक करके सारे दोगले चेहरे बेनकाब हो रहे है, फिर चाहे वो दिल्ली के केजरीवाल, उत्तर प्रदेश के अखिलेश या फिर बिहार के तेजस्वी यादव हो!! #JusticeForShahabuddin",1,0.188757,False
157,"शुक्रिया…""यादव परिवार""…एक बार और साबित करने के लिए कि ""कुफ़्फ़ार-ला-ऐतबार"" #shahabuddinsaheb #JusticeForShahabuddin",1,0.214696,False
509,"RT @ColdCigar: ""इंजीनियर लोग शायद सड़क पे डिवाइडर बनाना भूल गए, देश के लिये मैं खुद ही खड़ा हो जाता हूँ।"" #DividerInChief https://t.co/zFb…",1,0.245817,False
241,"#ModiKaVaccineJumla इन #हरामियों में से अगर कोई #सेवा करता दिख रहा हो तो #बताना, उसे #पुरस्कार देना है https://t.co/nswsPovJZn",1,0.247852,False
377,"Thanks God;सऊदी से आए टैंकर सही सलामत सऊदी पहुँच गए,नही तो ""Mr बेचू"" उसका भी सौदा कर देता #OxygenShortage #OxygenBeds #ModiKaVaccineJumla #ModiOxygenDo #ModiHataoDeshBachao #ModiSarkarHiSystemHai",1,0.264539,False
471,RT @1012ats: देखो अपने 150% इंटेलिजेंस Einstein की छठी औलाद को Ms Sonam Jain\n@sonamJain001\n@ShilpiSinghINC https://t.co/Tc1LngLjch,1,0.282392,False
262,"तुम्हारा जुल्म काफी है हमें बेदार करने को, जो लोहा चोट खाता है वही तलवार बनता है। #एक_बनो_नेक_बनो #JusticeForShahabuddin #",1,0.300880,False
126,"RT @barotchetan99: पूरी दुनिया हैरान है ये जानकर भारत सब्सिडी पर सूअर पालता है.. \nJNU , AMU",1,0.326204,False


In [ ]:
error_analysis_hindi = pd.concat([error_analysis_hindi , pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1).text], axis = 0)

In [ ]:
# Model is unsure, predicted prob is abs(predicted_prob - 0.5)

pd.set_option('display.max_colwidth', None)


temp_df = Analysis.predicted_prob.sub(0.5).abs().nsmallest(15)

pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df], axis = 1).rename(columns={'predicted_prob':'abs(predicted_prob - 0.5)'})


,text,label,abs(predicted_prob - 0.5)
431,"@TejYadav14 शहाबुद्दीन की जगह लालू यादव की "" संस्थानिक हत्या "" हुई होती तो ससम्मान अंतिम संस्कार के लिए मुसलमान हॉस्पिटल में छलांग लगा दिए होते एक बच्चा ( ओसामा ) अपने पिता का ससम्मान अंतिम संस्कार के लिए कोर्ट से हॉस्पिटल तक दर दर भटक रहा और लालू परिवार तमाशा देख रहा #JusticeForShahabuddin",1,0.000167
506,"RT @sanjuydv: सुनो पाइलट,\n\nरडार से बचने के लिए तुम प्लेन को बादल की स्पीड से चलाना,मैं बादल को प्लेन की स्पीड से चला दूँगा।\n\nऔर जानते हो?…",0,0.000296
89,RT @de_real_mak: @RoshanKrRai @DeanOfJio @sudhirchaudhary अगर 2000 का नोट मुस्लिम के घर में हुआ तो सिग्नल पकड़ेगा और हिंदू के घर हुआ तो न…,1,0.000638
476,"RT @HandsomAbhishek: रामचरित मानस में साफ लिखा है.......\n\n""जब जब होहिं धरम पर हानि\nबाढ़ाहिं असुर अधम अभिमानी\nतब तब प्रभु धरि मनुज शरीरा\nहर…",0,0.000864
272,kisan ekta zindabaad #RIPAjitSingh #FarmersProtest #ModiResign #ModiMustResign #ModiKaVaccineJumla #मोदी_इस्तीफा_दो,0,0.001159
326,@hadappababa आजाएंगे हमें छोड़ कर कहा जाएंगे 🙈,0,0.002100
517,#bengalviolence Be careful everyone. Some unscrupulous people are posting these pictures on social media to spread tension in the state and the country. #CovidIndia #COVIDEmergencyIndia #ममता_की_निर्ममता #SpinelessBJP #SpinlessBJP #बंगाल_हिंसा #COVIDEmergency #BengalBurning https://t.co/s1A7UpaBju,0,0.002212
387,मोदी को लगता है कि वह PM की कुर्शी पर बैठा है और हकीकत यह है कि वो जनता की ला'शो के ढेर पर बैठकर सत्ता का सुख भोग।रहा है.!! #ResignModi 😡😡😡,1,0.002317
249,"RT @manuazad_: दिल्ली के लोगों इतनी झाडू चला दो, इतनी झाडू चला कि जुमलों की गंदगी मचाने वाले दिल्ली से बाहर निकल कर भागे। \n#Vote_4_Aap✌️✌️…",0,0.003315
419,Ye bumrah lodu phir G maar dega,1,0.003570


In [ ]:
error_analysis_hindi = pd.concat([error_analysis_hindi, pd.concat([hindi_test.iloc[list(temp_df.index)], temp_df], axis = 1).text], axis = 0)

In [ ]:
error_analysis_hindi.to_csv('error_analysis_hindi.csv', index=False)

In [ ]:
!ls

error_analysis_hindi.csv  gdrive  sample_data


###Hindi Zero-Shot and Transfer Learning

In [ ]:
# reload saved model for English Pretrained

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_For_Hindi+Korean/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rate = .001
dropout = 0
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

# #Results on English
# y_pred = reloaded_model.predict(x=[english_x_dev.input_ids, english_x_dev.token_type_ids, english_x_dev.attention_mask])

# print('Dev Set Macro F-1 Score (on English Data): {:.3f}'.format(f1_score(english_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

# y_pred = reloaded_model.predict(x=[english_x_test.input_ids, english_x_test.token_type_ids, english_x_test.attention_mask])

# print('Test Set Macro F-1 Score (on English Data: {:.3f}'.format(f1_score(english_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))

#Results on Hindi (Zero-Shot)
y_pred = reloaded_model.predict(x=[hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask])

print('Dev Set F-1 Score (on Hindi Data (Zero-Shot)): {:.3f}'.format(f1_score(hindi_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score (on Hindi Data (Zero-Shot)): {:.3f}'.format(f1_score(hindi_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[hindi_x_test.input_ids, hindi_x_test.token_type_ids, hindi_x_test.attention_mask])

print('Test Set F-1 Score (on Hindi Data (Zero-Shot)): {:.3f}'.format(f1_score(hindi_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score (on Hindi Data (Zero-Shot)): {:.3f}'.format(f1_score(hindi_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))


Dev Set F-1 Score (on Hindi Data (Zero-Shot)): 0.441
Dev Set Macro F-1 Score (on Hindi Data (Zero-Shot)): 0.519
Test Set F-1 Score (on Hindi Data (Zero-Shot)): 0.413
Test Set Macro F-1 Score (on Hindi Data (Zero-Shot)): 0.513


In [ ]:
# reload saved model for Hindi (mBERT transfer learning)

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_Hindi_Finetuned/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rate = .001
dropout = 0
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

y_pred = reloaded_model.predict(x=[hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask])

print('Dev Set F-1 Score (on Hindi Data (Transfer Learning)): {:.3f}'.format(f1_score(hindi_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score (on Hindi Data (Transfer Learning)): {:.3f}'.format(f1_score(hindi_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[hindi_x_test.input_ids, hindi_x_test.token_type_ids, hindi_x_test.attention_mask])

print('Test Set F-1 Score (on Hindi Data (Transfer Learning)): {:.3f}'.format(f1_score(hindi_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score (on Hindi Data (Transfer Learning)): {:.3f}'.format(f1_score(hindi_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))


Dev Set F-1 Score (on Hindi Data (Transfer Learning)): 0.725
Dev Set Macro F-1 Score (on Hindi Data (Transfer Learning)): 0.608
Test Set F-1 Score (on Hindi Data (Transfer Learning)): 0.721
Test Set Macro F-1 Score (on Hindi Data (Transfer Learning)): 0.610


###Korean Baseline mBERT

In [70]:
# reload saved model for Korean (baseline mBERT)

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/Korean_mBERT_Baseline/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with korean
random.seed(42)
learning_rate = .001
dropout = 0.15
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

y_pred = reloaded_model.predict(x=[korean_x_dev.input_ids, korean_x_dev.token_type_ids, korean_x_dev.attention_mask])

print('Dev Set F-1 Score: {:.3f}'.format(f1_score(korean_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score: {:.3f}'.format(f1_score(korean_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[korean_x_test.input_ids, korean_x_test.token_type_ids, korean_x_test.attention_mask])

print('Test Set F-1 Score: {:.3f}'.format(f1_score(korean_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score: {:.3f}'.format(f1_score(korean_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))

Dev Set F-1 Score: 0.713
Dev Set Macro F-1 Score: 0.628
Test Set F-1 Score: 0.700
Test Set Macro F-1 Score: 0.639


In [72]:
pd.DataFrame({'mBERT_predicted_prob':np.concatenate(y_pred), 
              'predicted label':np.concatenate(y_pred>0.5).astype(int),
              'TP_FP_mBERT':TP_FP(korean_y_test, np.concatenate(y_pred>0.5).astype(int))}).to_excel('korean.xlsx')

###Extract Examples for Error Analysis

In [ ]:
Analysis = pd.DataFrame(data = {'predicted_prob':y_pred.reshape(korean_y_test.shape), 'predicted_class':(y_pred>0.5).reshape(korean_y_test.shape).astype(int),'true_class':korean_y_test})

In [ ]:
Analysis

,predicted_prob,predicted_class,true_class
0,0.738573,1,0
1,0.538453,1,0
2,0.721100,1,1
3,0.583360,1,0
4,0.534640,1,1
...,...,...,...
725,0.511468,1,0
726,0.368835,0,0
727,0.375134,0,0
728,0.572300,1,0


In [ ]:
Analysis['correct_prediction'] = Analysis.predicted_class == Analysis.true_class

In [ ]:
Analysis.reset_index(drop=True, inplace=True)

In [ ]:
Analysis

,predicted_prob,predicted_class,true_class,correct_prediction
0,0.738573,1,0,False
1,0.538453,1,0,False
2,0.721100,1,1,True
3,0.583360,1,0,False
4,0.534640,1,1,True
...,...,...,...,...
725,0.511468,1,0,False
726,0.368835,0,0,True
727,0.375134,0,0,True
728,0.572300,1,0,False


In [ ]:
# Correctly classified as toxic with the largest probability
pd.set_option('display.max_colwidth', None)

temp_df = Analysis[(Analysis.correct_prediction==True) & (Analysis.true_class == 1)].nlargest(15, ['predicted_prob'])

pd.concat([korean_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

,text,label,predicted_prob,correct_prediction
237,그냥 무작정 까고보네 미성년자도 결혼할수있는거고 나이처먹고도 애지우려드는 너희들보다 훨 떳떳하구만 지금도 비방댓글 달면서 부모님 등골뽑아먹는것들이 부들대는거봐라 인생이 늘 불만이네,1,0.814815,True
109,임현주한테 열폭부리는 것들 많넼ㅋㅋ 하긴 지들은 임현주가 가진 매력의 반의반도없으니 남자못꼬시지,1,0.804288,True
104,죄앙이 딱새 교익아 좀배워라인생을어케사는줄을그리구돈이전부가아니란걸사회 민심이어느쪽으로흘러가는가 를,1,0.803923,True
56,실명을 못깔정도로 증거도 없고 이슈몰이에만 급급해서 일단 자극적으로 왜곡 편집 했겠지. 그걸 믿는거고 뇌에 우동사리든 방구석 여포들은 ㅋ 뭐 어디 국과수에서 양성반응 나왔어? 그알이 곧 법이고 진실이여? ㅋㅋㅋㅋ,1,0.803212,True
464,돼지 뭐 어쩌라구‼️‼️‼️ 뭘 가득 처먹고 그러냐 아프리카는 마실 물도 없는데 가득 처먹는거 잡아 없애야지‼️‼️⁉️⁉️,1,0.794636,True
53,걍 부러우면 부럽다 해라. 뭘 돈이면 다 된다고 비꼬냐? 월 300 받을라고 사장 똥꼬도 빠는 것들이ㅋㅋ,1,0.787977,True
135,뽕빤넘들 처럼 생기고 말하는거 보니까 양아치스럽던데 그걸 멋으로 알거아냐?ㅋㅋㅋ잔나비인지 잔잔바리인지 뭐시깽이들 꺼져라,1,0.783264,True
153,남탓 하는것도 정신병이고 저질국민 성이다.. 그냥 고인에 명복이나 빌어르 어떻게 누구 하나 죽었다하면 죄다 남탓이냐? 자식죽은걸로 지가 뭐나 된양 정치질 하려는 부모들도 역겹다,1,0.782432,True
636,이쁘네 ㅋㅋㅋ 오크먜갈년들 또 심통난거냐 ? 왜 ㅂㄷㅂㄷ 욕하고난리냐 ㅋㅋㅋ 하긴 니네면상은 견적이안나오니 ㅋㅋㅋㅋ,1,0.781768,True
312,"돼조이 얼굴 개빻았고 표정은물론 발성,발음 다 질질새고 눈 게슴츠레뜨는거 극혐임",1,0.778507,True


In [ ]:
error_analysis_korean  = pd.concat([korean_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1).text

In [ ]:
# Correctly classified as non-toxic with the lowest probability (of toxic)
pd.set_option('display.max_colwidth', None)

temp_df = Analysis[(Analysis.correct_prediction==True) & (Analysis.true_class == 0)].nsmallest(15, ['predicted_prob'])

pd.concat([korean_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

,text,label,predicted_prob,correct_prediction
299,힘내세요~ 응원합니다!!,0,0.114138,True
276,엄마 닮아서 길쭉하니 이쁘네요♡♡♡,0,0.135878,True
454,이 드라마 아직도 하네~~~,0,0.151818,True
527,저 진짜 기다렸어요 ㅠㅠㅠ 눈물 나오네 ㅠㅠㅠㅠ 사랑해요!!!,0,0.163003,True
137,솔직히 좀 빠졌으면 좋겠어요,0,0.168306,True
518,신선해서 좋았음~!!!,0,0.168913,True
268,애기랑 꼭 나와주세요. 나~중에요^^,0,0.191582,True
18,사진은 수지닮았네 이뻐요~,0,0.203849,True
525,자식은 축복이에요~힘내세요~^^,0,0.208601,True
337,이제 웃기기만 하면 된다~,0,0.208736,True


In [ ]:
error_analysis_korean  = pd.concat([error_analysis_korean, pd.concat([korean_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1).text], axis = 0)

In [ ]:
# incorrectly classified as toxic with the highest probability (of toxic)
pd.set_option('display.max_colwidth', None)


temp_df = Analysis[(Analysis.correct_prediction==False) & (Analysis.true_class == 0)].nlargest(15, ['predicted_prob'])

pd.concat([korean_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

,text,label,predicted_prob,correct_prediction
43,문식이는 엄마를 뺏기는게 두려운거아닐까요..버려질까봐 무서운거구요..사랑이 고픈 어린아이같아요..친아들이었다면 저러지않았겠지요..친부모도 자길 버렸는데 새엄마가 날 언제 내칠까하는 두려움이 컷을듯해요..,0,0.769910,False
532,송종국이보이네 ㅋ 역시 남자는 머리빨,0,0.754629,False
340,ㅋㅋㅋㅋㅋㅋ마 서빙은 누가하노 정신차리고 퍼뜩온나,0,0.740840,False
0,"팀으로 데뷔한거면 개인활동 했어도 N빵 해야지... 그게 팀을 위해서도 맞는거고~~ 근데 여기 댓글은 다 왜 나누냐는 식이네, 당장 혼자만 배불러야 된다는 심리들인건지, 팀이 유지되야 인기도 오래갈텐데",0,0.738573,False
171,아직 잘 몰겠음 무슨 속사정이있을듯 넘 맘이아파야 여자하나 잘못만나 그나이에 어째여,0,0.738394,False
389,종국이가 낫구만 얼굴이 다가아니다 사람은,0,0.733982,False
705,왜 못 잡아먹어서 안달이야 걍 냅둬라 좀,0,0.732916,False
691,딸 예뻤는데??뭐징?,0,0.728688,False
217,먼가 내용 점점 산으로 가는듯 처음의 신선함은 없어지고 질질 끄는 듯한 기분 보는 내내 폰 만지작 거렸어용,0,0.722852,False
481,박보검 머리 기르니까 뷔도 머리기르고 송중기까지 머리기르넨ㄴㅌㅌㅌㅌㅌㅌㅌ,0,0.722568,False


In [ ]:
error_analysis_korean  = pd.concat([error_analysis_korean, pd.concat([korean_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1).text], axis = 0)

In [ ]:
# incorrectly classified as non-toxic with the lowest probability (of toxic)
pd.set_option('display.max_colwidth', None)


temp_df = Analysis[(Analysis.correct_prediction==False) & (Analysis.true_class == 1)].nsmallest(15, ['predicted_prob'])

pd.concat([korean_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

,text,label,predicted_prob,correct_prediction
229,예 가수야?!,1,0.176524,False
41,우익사쿠라 한국에서 꺼져줘~,1,0.208516,False
347,와 나이많아도 임신이 되는구나 축하!!!,1,0.236497,False
722,이제 군대 피해의식에서 좀 벗어들 나시지요~,1,0.237340,False
633,스티브 유기사!!! 당장 내리시오!!보기 싫다,1,0.270781,False
500,일본 원숭이는 너네 나라로 가세요!,1,0.279490,False
19,근데 솔직히 김정훈 천재유전자 받고 싶어하는 여자들 많을텐데..,1,0.282290,False
225,장동윤 ㅅㅅ 잘 할 것 같음,1,0.289513,False
593,애기 진짜 안이쁘다...,1,0.307079,False
6,남자가 사회 생활하다 보면 손이 스칠수도 있는거지 거참~~,1,0.330085,False


In [ ]:
error_analysis_korean  = pd.concat([error_analysis_korean, pd.concat([korean_test.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1).text], axis = 0)

In [ ]:
# Model is unsure, predicted prob is abs(predicted_prob - 0.5)

pd.set_option('display.max_colwidth', None)


temp_df = Analysis.predicted_prob.sub(0.5).abs().nsmallest(15)

pd.concat([korean_test.iloc[list(temp_df.index)], temp_df], axis = 1).rename(columns={'predicted_prob':'abs(predicted_prob - 0.5)'})


,text,label,abs(predicted_prob - 0.5)
700,나같아도 지울꺼같긴하다 나쁜놈,1,0.000074
270,정을 주고 싶어도 희한하게 정이 안 가는 나영석 사단,0,0.000173
120,한혜진씨 의상은 쫌 불편했습니다 ㅠㅠ,0,0.000430
190,아들이 뭔잘못이여....마이크로닷은 잘못이 없습니다,0,0.000465
287,에구 절대 말 썩어서 득이 없는 인간을 대하는 임블리 힘내세요.,0,0.000900
206,비지니스 그룹,1,0.001786
663,19에서15로 바뀌고 망트리 탄거...,0,0.002330
288,존잘 존예,0,0.002349
635,자살은 하지마시길...,1,0.003290
310,티아라 명칭은 전소속사 소유가 맞다,0,0.003508


In [ ]:
error_analysis_korean  = pd.concat([error_analysis_korean, pd.concat([korean_test.iloc[list(temp_df.index)], temp_df], axis = 1).text], axis = 0)

In [ ]:
error_analysis_korean.to_csv('error_analysis_korean.csv', index=False)

In [ ]:
!ls

error_analysis_korean.csv  gdrive  sample_data


In [ ]:
from google.colab import files
files.download('error_analysis_hindi.csv') 
files.download('error_analysis_korean.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Korean Zero-Shot and Transfer Learning

In [ ]:
# reload saved model for English Pretrained

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_For_Hindi+Korean/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rate = .001
dropout = 0
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

# #Results on English
# y_pred = reloaded_model.predict(x=[english_x_dev.input_ids, english_x_dev.token_type_ids, english_x_dev.attention_mask])

# print('Dev Set Macro F-1 Score (on English Data): {:.3f}'.format(f1_score(english_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

# y_pred = reloaded_model.predict(x=[english_x_test.input_ids, english_x_test.token_type_ids, english_x_test.attention_mask])

# print('Test Set Macro F-1 Score (on English Data: {:.3f}'.format(f1_score(english_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))

#Results on Korean (Zero-Shot)
y_pred = reloaded_model.predict(x=[korean_x_dev.input_ids, korean_x_dev.token_type_ids, korean_x_dev.attention_mask])

print('Dev Set F-1 Score (on Korean Data (Zero-Shot)): {:.3f}'.format(f1_score(korean_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score (on Korean Data (Zero-Shot)): {:.3f}'.format(f1_score(korean_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[korean_x_test.input_ids, korean_x_test.token_type_ids, korean_x_test.attention_mask])

print('Test Set F-1 Score (on Korean Data (Zero-Shot)): {:.3f}'.format(f1_score(korean_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score (on Korean Data (Zero-Shot)): {:.3f}'.format(f1_score(korean_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))


Dev Set F-1 Score (on Korean Data (Zero-Shot)): 0.610
Dev Set Macro F-1 Score (on Korean Data (Zero-Shot)): 0.602
Test Set F-1 Score (on Korean Data (Zero-Shot)): 0.580
Test Set Macro F-1 Score (on Korean Data (Zero-Shot)): 0.578


In [ ]:
# reload saved model for Korean (mBERT transfer learning)

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_Korean_Finetuned/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rate = .001
dropout = 0.15
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

y_pred = reloaded_model.predict(x=[korean_x_dev.input_ids, korean_x_dev.token_type_ids, korean_x_dev.attention_mask])

print('Dev Set F-1 Score (on Korean Data (Transfer Learning)): {:.3f}'.format(f1_score(korean_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score (on Korean Data (Transfer Learning)): {:.3f}'.format(f1_score(korean_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[korean_x_test.input_ids, korean_x_test.token_type_ids, korean_x_test.attention_mask])

print('Test Set F-1 Score (on Korean Data (Transfer Learning)): {:.3f}'.format(f1_score(korean_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score (on Korean Data (Transfer Learning)): {:.3f}'.format(f1_score(korean_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))

Dev Set F-1 Score (on Korean Data (Transfer Learning)): 0.727
Dev Set Macro F-1 Score (on Korean Data (Transfer Learning)): 0.682
Test Set F-1 Score (on Korean Data (Transfer Learning)): 0.700
Test Set Macro F-1 Score (on Korean Data (Transfer Learning)): 0.665


##Reload Model and Test on Translated Data

###Translated Hindi Zero-Shot and Transfer Learning

In [ ]:
# reload saved model for English Pretrained

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_For_Hindi+Korean/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rate = .001
dropout = 0
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

#Results on English
# y_pred = reloaded_model.predict(x=[english_x_dev.input_ids, english_x_dev.token_type_ids, english_x_dev.attention_mask])

# print('Dev Set Macro F-1 Score (on English Data): {:.3f}'.format(f1_score(english_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

# y_pred = reloaded_model.predict(x=[english_x_test.input_ids, english_x_test.token_type_ids, english_x_test.attention_mask])

# print('Test Set Macro F-1 Score (on English Data: {:.3f}'.format(f1_score(english_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))

#Results on Hindi (Zero-Shot)
y_pred = reloaded_model.predict(x=[hindi_translated_x_dev.input_ids, hindi_translated_x_dev.token_type_ids, hindi_translated_x_dev.attention_mask])

print('Dev Set F-1 Score (on Translated Hindi Data (Zero-Shot)): {:.3f}'.format(f1_score(hindi_translated_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score (on Translated Hindi Data (Zero-Shot)): {:.3f}'.format(f1_score(hindi_translated_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[hindi_translated_x_test.input_ids, hindi_translated_x_test.token_type_ids, hindi_translated_x_test.attention_mask])

print('Test Set F-1 Score (on Translated Hindi Data (Zero-Shot)): {:.3f}'.format(f1_score(hindi_translated_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score (on Translated Hindi Data (Zero-Shot)): {:.3f}'.format(f1_score(hindi_translated_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))


Dev Set F-1 Score (on Translated Hindi Data (Zero-Shot)): 0.347
Dev Set Macro F-1 Score (on Translated Hindi Data (Zero-Shot)): 0.494
Test Set F-1 Score (on Translated Hindi Data (Zero-Shot)): 0.380
Test Set Macro F-1 Score (on Translated Hindi Data (Zero-Shot)): 0.517


In [ ]:
# reload saved model for Hindi (mBERT transfer learning)

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_TranslatedHindi_FineTuned/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rate = .001
dropout = 0
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

y_pred = reloaded_model.predict(x=[hindi_translated_x_dev.input_ids, hindi_translated_x_dev.token_type_ids, hindi_translated_x_dev.attention_mask])

print('Dev Set F-1 Score (on Translated Hindi Data (Transfer Learning)): {:.3f}'.format(f1_score(hindi_translated_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score (on Translated Hindi Data (Transfer Learning)): {:.3f}'.format(f1_score(hindi_translated_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[hindi_x_test.input_ids, hindi_x_test.token_type_ids, hindi_x_test.attention_mask])

print('Test Set F-1 Score (on Translated Hindi Data (Transfer Learning)): {:.3f}'.format(f1_score(hindi_translated_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score (on Translated Hindi Data (Transfer Learning)): {:.3f}'.format(f1_score(hindi_translated_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))


Dev Set F-1 Score (on Translated Hindi Data (Transfer Learning)): 0.717
Dev Set Macro F-1 Score (on Translated Hindi Data (Transfer Learning)): 0.665
Test Set F-1 Score (on Translated Hindi Data (Transfer Learning)): 0.707
Test Set Macro F-1 Score (on Translated Hindi Data (Transfer Learning)): 0.654


###Translated Korean Zero-Shot and Transfer Learning

In [ ]:
# reload saved model for English Pretrained

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_For_Hindi+Korean/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rate = .001
dropout = 0
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

#Results on English
# y_pred = reloaded_model.predict(x=[english_x_dev.input_ids, english_x_dev.token_type_ids, english_x_dev.attention_mask])

# print('Dev Set Macro F-1 Score (on English Data): {:.3f}'.format(f1_score(english_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

# y_pred = reloaded_model.predict(x=[english_x_test.input_ids, english_x_test.token_type_ids, english_x_test.attention_mask])

# print('Test Set Macro F-1 Score (on English Data: {:.3f}'.format(f1_score(english_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))

#Results on Korean (Zero-Shot)
y_pred = reloaded_model.predict(x=[korean_translated_x_dev.input_ids, korean_translated_x_dev.token_type_ids, korean_translated_x_dev.attention_mask])

print('Dev Set F-1 Score (on Translated Korean Data (Zero-Shot)): {:.3f}'.format(f1_score(korean_translated_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score (on Translated Korean Data (Zero-Shot)): {:.3f}'.format(f1_score(korean_translated_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[korean_translated_x_test.input_ids, korean_translated_x_test.token_type_ids, korean_translated_x_test.attention_mask])

print('Test Set F-1 Score (on Translated Korean Data (Zero-Shot)): {:.3f}'.format(f1_score(korean_translated_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score (on Translated Korean Data (Zero-Shot)): {:.3f}'.format(f1_score(korean_translated_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))


Dev Set F-1 Score (on Translated Korean Data (Zero-Shot)): 0.571
Dev Set Macro F-1 Score (on Translated Korean Data (Zero-Shot)): 0.592
Test Set F-1 Score (on Translated Korean Data (Zero-Shot)): 0.542
Test Set Macro F-1 Score (on Translated Korean Data (Zero-Shot)): 0.589


In [ ]:
# reload saved model for Korean (mBERT transfer learning)

#Define checkpoint directory
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks (1)/W266_Final_Model_CheckPoints/English_Pretrained_TranslatedKorean_FineTuned/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#hyperparameter tuning, only choose the best ones found with hindi
random.seed(42)
learning_rate = .001
dropout = 0.15
hidden_size = (256, 128)
train_layer = 0


reloaded_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(learning_rate), 
                                                    train_layers = train_layer,
                                                    hidden_initializer = 'he_uniform',
                                                    dropout = dropout,
                                                    hidden_sizes = hidden_size
                                                    )

# Loads the weights
reloaded_model.load_weights(checkpoint_path)

y_pred = reloaded_model.predict(x=[korean_translated_x_dev.input_ids, korean_translated_x_dev.token_type_ids, korean_translated_x_dev.attention_mask])

print('Dev Set F-1 Score (on Translated Korean Data (Transfer Learning)): {:.3f}'.format(f1_score(korean_translated_y_dev, np.concatenate(y_pred>0.5).astype(int))))
print('Dev Set Macro F-1 Score (on Translated Korean Data (Transfer Learning)): {:.3f}'.format(f1_score(korean_translated_y_dev, np.concatenate(y_pred>0.5).astype(int), average='macro')))

y_pred = reloaded_model.predict(x=[korean_translated_x_test.input_ids, korean_translated_x_test.token_type_ids, korean_translated_x_test.attention_mask])

print('Test Set F-1 Score (on Translated Korean Data (Transfer Learning)): {:.3f}'.format(f1_score(korean_translated_y_test, np.concatenate(y_pred>0.5).astype(int))))
print('Test Set Macro F-1 Score (on Translated Korean Data (Transfer Learning)): {:.3f}'.format(f1_score(korean_translated_y_test, np.concatenate(y_pred>0.5).astype(int), average='macro')))

Dev Set F-1 Score (on Translated Korean Data (Transfer Learning)): 0.714
Dev Set Macro F-1 Score (on Translated Korean Data (Transfer Learning)): 0.663
Test Set F-1 Score (on Translated Korean Data (Transfer Learning)): 0.689
Test Set Macro F-1 Score (on Translated Korean Data (Transfer Learning)): 0.656


#Previous Code

In [ ]:
Analysis['correct_prediction'] = Analysis.predicted_class == Analysis.true_class

In [ ]:
Analysis.reset_index(drop=True, inplace=True)

In [ ]:
# Correctly classified as toxic with the largest probability
pd.set_option('display.max_colwidth', None)

temp_df = Analysis[(Analysis.correct_prediction==True) & (Analysis.true_class == 1)].nlargest(25, ['predicted_prob'])

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

In [ ]:
# Correctly classified as non-toxic with the lowest probability (of toxic)
pd.set_option('display.max_colwidth', None)

temp_df = Analysis[(Analysis.correct_prediction==True) & (Analysis.true_class == 0)].nsmallest(25, ['predicted_prob'])

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

In [ ]:
# incorrectly classified as toxic with the highest probability (of toxic)
pd.set_option('display.max_colwidth', None)


temp_df = Analysis[(Analysis.correct_prediction==False) & (Analysis.true_class == 0)].nlargest(25, ['predicted_prob'])

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

In [ ]:
# incorrectly classified as non-toxic with the lowest probability (of toxic)
pd.set_option('display.max_colwidth', None)


temp_df = Analysis[(Analysis.correct_prediction==False) & (Analysis.true_class == 1)].nsmallest(25, ['predicted_prob'])

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df[['predicted_prob', 'correct_prediction']]], axis = 1)

In [ ]:
# Model is unsure, predicted prob is abs(predicted_prob - 0.5)

pd.set_option('display.max_colwidth', None)


temp_df = Analysis.predicted_prob.sub(0.5).abs().nsmallest(25)

pd.concat([hindi_df_dev.iloc[list(temp_df.index)], temp_df], axis = 1).rename(columns={'predicted_prob':'abs(predicted_prob - 0.5)'})


In [ ]:
outputs = mbert_model(hindi_x_dev.input_ids[:5], hindi_x_dev.token_type_ids[:5], hindi_x_dev.attention_mask[:5], output_attentions=True )

In [ ]:
outputs